## Further text preprocessing

### Pre-requisites: downloading nltk Dutch stopwords and SpaCy model

In [4]:
import nltk
nltk.download('stopwords', 'dutch')

[nltk_data] Downloading package stopwords to dutch...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [6]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [7]:
# SpaCy for lemmatization
import spacy

In [8]:
# Plotting tools
!pip install pyLDAvis
###!pip install gensimvis.py

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
###import pyLDAvis.gensim as gensimvis

import matplotlib.pyplot as plt

Le chemin d'acc�s sp�cifi� est introuvable.


In [9]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

### Prepare stopwords

In [10]:
# NLTK Stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

#stop_words = stopwords.words('dutch')
stop_words = nltk.corpus.stopwords.words('dutch')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrateur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Tokenize words and clean-up text

In [11]:
# Importing file handling library
import os

file = open('./numac=2019041722.txt','rt')
raw_text = file.read()


In [12]:
###print(raw_text)

In [13]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize
print(word_tokenize(raw_text))

['VERSLAG', 'AAN', 'DE', 'KONINGSire', ',', 'Het', 'ontwerp', 'van', 'koninklijk', 'besluit', 'dat', 'wij', 'de', 'eer', 'hebben', 'aan', 'uwe', 'Majesteit', 'voor', 'te', 'leggen', ',', 'beoogt', 'de', 'uitvoering', 'van', 'de', 'artikelen', '93ter', 'tot', '93quinquies', 'van', 'het', 'Wetboek', 'van', 'de', 'belasting', 'over', 'de', 'toegevoegde', 'waarde', '(', 'hierna', "''", 'Wbtw', "''", ')', ',', 'de', 'artikelen', '412bis', ',', '433', 'tot', '435', 'van', 'het', 'Wetboek', 'van', 'de', 'inkomstenbelastingen', '1992', '(', 'hierna', '``', 'WIB', '92', "''", ')', ',', 'de', 'artikelen', '35', 'tot', '37', 'en', '43', 'tot', '45', 'en', '47', 'van', 'het', 'Wetboek', 'van', 'de', 'minnelijke', 'en', 'gedwongen', 'invordering', 'van', 'fiscale', 'en', 'niet-fiscale', 'schuldvorderingen', '(', 'hierna', '``', 'Invorderingswetboek', "''", ')', 'en', 'de', 'artikelen', '157', 'tot', '159', 'en', '161', 'van', 'de', 'programmawet', '(', 'I', ')', 'van', '29', 'maart', '2012', '(', '

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrateur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
###raw_text = re.sub('[^a-z\ \']+', " ", raw_text)
###data_words = list(raw_text.split())
###print(data_words)

In [15]:
# Tokenizing each sentence into a list of words, removing punctuations and unnecessary characters altogether.
# Gensim’s simple_preprocess() is the best utility for this . Additionally 'deacc=True' removes the punctuations.

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

#data_words = list(sent_to_words(raw_text))
data_words = list(word_tokenize(raw_text))
print(data_words)


['VERSLAG', 'AAN', 'DE', 'KONINGSire', ',', 'Het', 'ontwerp', 'van', 'koninklijk', 'besluit', 'dat', 'wij', 'de', 'eer', 'hebben', 'aan', 'uwe', 'Majesteit', 'voor', 'te', 'leggen', ',', 'beoogt', 'de', 'uitvoering', 'van', 'de', 'artikelen', '93ter', 'tot', '93quinquies', 'van', 'het', 'Wetboek', 'van', 'de', 'belasting', 'over', 'de', 'toegevoegde', 'waarde', '(', 'hierna', "''", 'Wbtw', "''", ')', ',', 'de', 'artikelen', '412bis', ',', '433', 'tot', '435', 'van', 'het', 'Wetboek', 'van', 'de', 'inkomstenbelastingen', '1992', '(', 'hierna', '``', 'WIB', '92', "''", ')', ',', 'de', 'artikelen', '35', 'tot', '37', 'en', '43', 'tot', '45', 'en', '47', 'van', 'het', 'Wetboek', 'van', 'de', 'minnelijke', 'en', 'gedwongen', 'invordering', 'van', 'fiscale', 'en', 'niet-fiscale', 'schuldvorderingen', '(', 'hierna', '``', 'Invorderingswetboek', "''", ')', 'en', 'de', 'artikelen', '157', 'tot', '159', 'en', '161', 'van', 'de', 'programmawet', '(', 'I', ')', 'van', '29', 'maart', '2012', '(', '

### Creating bigrams and trigrams

In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words]])

['VERSLAG', 'AAN', 'DE', 'KONINGSire', ',', 'Het', 'ontwerp', 'van', 'koninklijk', 'besluit', 'dat', 'wij', 'de', 'eer', 'hebben', 'aan', 'uwe', 'Majesteit', 'voor', 'te', 'leggen', ',', 'beoogt', 'de', 'uitvoering', 'van', 'de', 'artikelen', '93ter', 'tot', '93quinquies', 'van', 'het', 'Wetboek', 'van', 'de', 'belasting', 'over', 'de', 'toegevoegde', 'waarde', '(', 'hierna', "''", 'Wbtw', "''", ')', ',', 'de', 'artikelen', '412bis', ',', '433', 'tot', '435', 'van', 'het', 'Wetboek', 'van', 'de', 'inkomstenbelastingen', '1992', '(', 'hierna', '``', 'WIB', '92', "''", ')', ',', 'de', 'artikelen', '35', 'tot', '37', 'en', '43', 'tot', '45', 'en', '47', 'van', 'het', 'Wetboek', 'van', 'de', 'minnelijke', 'en', 'gedwongen', 'invordering', 'van', 'fiscale', 'en', 'niet-fiscale', 'schuldvorderingen', '(', 'hierna', '``', 'Invorderingswetboek', "''", ')', 'en', 'de', 'artikelen', '157', 'tot', '159', 'en', '161', 'van', 'de', 'programmawet', '(', 'I', ')', 'van', '29', 'maart', '2012', '(', '

###  Remove Stopwords, Make Bigrams and Lemmatize

In [17]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

### Calling the functions in order

In [18]:
 # Remove Stop Words

data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'nl' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('nl_core_news_lg', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized)

[['verslag'], [], [], [], [], [], ['ontwerp'], [], ['koninklijk'], ['besluit'], [], [], [], ['eer'], [], [], ['uwe'], ['majesteit'], [], [], ['leggen'], [], ['beogen'], [], ['uitvoering'], [], [], ['artikel'], [], [], ['quinquie'], [], [], ['wetboek'], [], [], ['belasting'], [], [], ['toegevoegde'], ['waarde'], [], ['hierna'], [], ['wbtw'], [], [], [], [], ['artikel'], ['bis'], [], [], [], [], [], [], ['wetboek'], [], [], [], [], [], ['hierna'], [], [], [], [], [], [], [], ['artikel'], [], [], [], [], [], [], [], [], [], [], [], ['wetboek'], [], [], ['minnelijk'], [], ['dwingen'], ['invordering'], [], ['fiscaal'], [], ['fiscaal'], [], [], ['hierna'], [], [], [], [], [], [], ['artikel'], [], [], [], [], [], [], [], ['programma_wet'], [], [], [], [], [], [], [], [], ['hierna'], [], ['programma_wet'], [], [], [], ['wijzigen'], [], [], ['wet'], [], [], [], [], ['houdenen'], ['fiscaal'], [], ['_fraude'], ['bestrijden'], [], ['f_inanciã'], [], [], [], ['divers'], ['bepaling'], [], [], ['wet'

In [19]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus)

[[(0, 1)], [], [], [], [], [], [(1, 1)], [], [(2, 1)], [(3, 1)], [], [], [], [(4, 1)], [], [], [(5, 1)], [(6, 1)], [], [], [(7, 1)], [], [(8, 1)], [], [(9, 1)], [], [], [(10, 1)], [], [], [(11, 1)], [], [], [(12, 1)], [], [], [(13, 1)], [], [], [(14, 1)], [(15, 1)], [], [(16, 1)], [], [(17, 1)], [], [], [], [], [(10, 1)], [(18, 1)], [], [], [], [], [], [], [(12, 1)], [], [], [], [], [], [(16, 1)], [], [], [], [], [], [], [], [(10, 1)], [], [], [], [], [], [], [], [], [], [], [], [(12, 1)], [], [], [(19, 1)], [], [(20, 1)], [(21, 1)], [], [(22, 1)], [], [(22, 1)], [], [], [(16, 1)], [], [], [], [], [], [], [(10, 1)], [], [], [], [], [], [], [], [(23, 1)], [], [], [], [], [], [], [], [], [(16, 1)], [], [(23, 1)], [], [], [], [(24, 1)], [], [], [(25, 1)], [], [], [], [], [(26, 1)], [(22, 1)], [], [(27, 1)], [(28, 1)], [], [(29, 1)], [], [], [], [(30, 1)], [(31, 1)], [], [], [(25, 1)], [], [], [], [], [(26, 1)], [(32, 1)], [], [], [(12, 1)], [], [], [(13, 1)], [], [], [(14, 1)], [(15, 1)],

In [20]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus]


[[('verslag', 1)],
 [],
 [],
 [],
 [],
 [],
 [('ontwerp', 1)],
 [],
 [('koninklijk', 1)],
 [('besluit', 1)],
 [],
 [],
 [],
 [('eer', 1)],
 [],
 [],
 [('uwe', 1)],
 [('majesteit', 1)],
 [],
 [],
 [('leggen', 1)],
 [],
 [('beogen', 1)],
 [],
 [('uitvoering', 1)],
 [],
 [],
 [('artikel', 1)],
 [],
 [],
 [('quinquie', 1)],
 [],
 [],
 [('wetboek', 1)],
 [],
 [],
 [('belasting', 1)],
 [],
 [],
 [('toegevoegde', 1)],
 [('waarde', 1)],
 [],
 [('hierna', 1)],
 [],
 [('wbtw', 1)],
 [],
 [],
 [],
 [],
 [('artikel', 1)],
 [('bis', 1)],
 [],
 [],
 [],
 [],
 [],
 [],
 [('wetboek', 1)],
 [],
 [],
 [],
 [],
 [],
 [('hierna', 1)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('artikel', 1)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('wetboek', 1)],
 [],
 [],
 [('minnelijk', 1)],
 [],
 [('dwingen', 1)],
 [('invordering', 1)],
 [],
 [('fiscaal', 1)],
 [],
 [('fiscaal', 1)],
 [],
 [],
 [('hierna', 1)],
 [],
 [],
 [],
 [],
 [],
 [],
 [('artikel', 1)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('program

In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=2000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.140*"besluiten" + 0.096*"ontvanger" + 0.096*"akte" + 0.091*"houden" + '
  '0.071*"overmacht" + 0.049*"advies" + 0.041*"wettelijk" + 0.028*"geã" + '
  '0.028*"ntegreerd" + 0.019*"aan_duiden"'),
 (1,
  '0.338*"wijzigen" + 0.175*"quinquie" + 0.099*"op_nemen" + 0.029*"divers" + '
  '0.026*"akt" + 0.021*"transactie" + 0.021*"Parlement" + 0.018*"zeer" + '
  '0.009*"in_stellen" + 0.009*"elek_tronisch"'),
 (2,
  '0.327*"programma_wet" + 0.256*"waarde" + 0.096*"technisch" + 0.021*"intern" '
  '+ 0.018*"stellen" + 0.009*"integratie" + 0.009*"belgiã" + '
  '0.009*"verzekeren" + 0.009*"hoofd_doel" + 0.001*"ambtenaar"'),
 (3,
  '0.303*"uitvoering" + 0.177*"vervangen" + 0.090*"federaal" + 0.055*"zending" '
  '+ 0.052*"twee" + 0.052*"omwille" + 0.050*"kader" + 0.017*"identificatie" + '
  '0.007*"leveren" + 0.007*"leggen"'),
 (4,
  '0.157*"kennisgeving" + 0.133*"vermelding" + 0.112*"toepassing" + '
  '0.072*"persoon" + 0.068*"zin" + 0.057*"modelbe_richt" + 0.045*"buitenland" '
  '+ 0.045*"ha

In [23]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) 



Perplexity:  -8.571116304997538


In [24]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

c:\Users\Administrateur\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11    -0.137701 -0.047476       1        1  7.187509
18    -0.235935 -0.306432       2        1  6.999055
17    -0.288581  0.288338       3        1  6.246926
13    -0.004678 -0.000887       4        1  5.946641
19    -0.042948 -0.007640       5        1  5.339390
10     0.048573  0.005107       6        1  5.118383
3      0.024176  0.002855       7        1  5.000243
9      0.021667  0.002639       8        1  4.970058
4      0.053108  0.005471       9        1  4.932873
6      0.014182  0.001649      10        1  4.919980
12     0.035987  0.003974      11        1  4.891958
16     0.051497  0.005374      12        1  4.808175
7      0.055315  0.005851      13        1  4.636987
0      0.067236  0.006604      14        1  4.501125
15     0.017516  0.002113      15        1  4.485086
2      0.049710  0.005473      16        1  4.379893
5      0.064848  0.006633      17        1  4.144316
1      0.052789  0.005627      18        1  4.138998
14     0.074457  0.007189      19        1  3.956112
8      0.078780  0.007538      20        1  3.396290, topic_info=           Term       Freq      Total Category  logprob  loglift
10      artikel  74.000000  74.000000  Default   30.000  30.0000
3       besluit  60.000000  60.000000  Default   29.000  29.0000
189         één  41.000000  41.000000  Default   28.000  28.0000
2    koninklijk  40.000000  40.000000  Default   27.000  27.0000
12      wetboek  37.000000  37.000000  Default   26.000  26.0000
..          ...        ...        ...      ...      ...      ...
7        leggen   0.071721   1.483612  Topic20   -6.565   0.3530
8        beogen   0.071721   1.415796  Topic20   -6.565   0.3998
9    uitvoering   0.071721  23.614124  Topic20   -6.565  -2.4143
11     quinquie   0.071721  11.760453  Topic20   -6.565  -1.7172
12      wetboek   0.071721  37.166448  Topic20   -6.565  -2.8679

[972 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
140      18  0.447025    Parlement
27       19  0.422838      _fraude
191      14  0.452088   aan_duiden
152      12  0.649778   aangepaste
98        6  0.896808  aangetekend
...     ...       ...          ...
240      18  0.494344         zeer
99        7  0.795074      zending
138       9  0.844000          zin
125      14  0.626424       zorgen
189       8  0.966164          één

[232 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 19, 18, 14, 20, 11, 4, 10, 5, 7, 13, 17, 8, 1, 16, 3, 6, 2, 15, 9])

In [25]:
import little as lmw

mallet_path = 'C:\\Users\\Administrateur\\Desktop\\mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

AttributeError: module 'gensim.models' has no attribute 'wrappers'

In [ ]:
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
!pip install --upgrade gensim==3.8


In [2]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [3]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

NameError: name 'id2word' is not defined

In [4]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

NameError: name 'x' is not defined

In [5]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

NameError: name 'x' is not defined

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)